## Import Packages

In [ ]:
from struct import *
import os
import random

import numpy as np
import matplotlib.pyplot as plt
import numpy as np


input_file = os.path.abspath(".") + "/input/Bar_24x2_directions.txt"
output_file = os.path.abspath(".") + "/input/Bar_24x2_directions_xy.txt"

## Params to the data

In [ ]:
#Images 
sz = (304, 240) #Image size
img = np.zeros(sz)
tau = 5000 #Value to normalize in the exponential
delta_t = 1000 #Value to remove old events

## Useful functions
The function decrease is used to apply the exponential decay to the patch generated.
The function is_corner_threshold is used to detect whether the patch generated is a corner or not.

In [ ]:
def decrease (t_old, t_current):
    if t_old == 0:
        val_decrease = 0.
    else:
        val_decrease = round(np.exp(-(t_current - t_old)/tau),3)
    return val_decrease

def is_corner_threshold(patch_update):
    sum_down = 0
    sum_up = 0
    sum_right = 0
    sum_left = 0
    
    for i in range(11):
        if patch_update[i,0] > 0.4:
            sum_up += patch_update[i,0]
            
        if patch_update[i,11] > 0.4:
            sum_down += patch_update[i,11]
            
        if patch_update[0,i] > 0.4:
            sum_left += patch_update[0,i]    
        
        if patch_update[11,i] > 0.4:
            sum_right += patch_update[11,i]
    if (sum_down < 1.4 and sum_right < 1.4) or (sum_down < 1.4 and sum_left < 1.4) or (sum_up < 1.4 and sum_right < 1.4) or (sum_up < 1.4 and sum_left < 1.4):
        return True
    else:
        return False

## INPUT
The input file corresponds to a list of pixel with their spiking time stamp. 
In the following for-loop, the patches are created: 
    1. When a pixel spikes, a patch 12 x 12 around it is generated. This patch is composed of the last spiking time stamps of the neurons.
    2. Then the exponential decay is applied to this patch in order to generate the time surface.
    3. Finally, the function is_corner_threshold detects whether the patch corresponds to a corner or not.

In [ ]:
print('Input being prepared with New Corner Detection')

time_stamp = []
input_data = []
with open(input_file, 'r') as events:
    vec_decrease = np.vectorize(decrease)
    
    for ev in events:
        [ts, x, y, p] = map(int, ev.split())
        img[x,y] = ts
        patch = img[(x-6):(x+6), (y-6):(y+6)]
                  
        if patch.shape == (12, 12):
            patch_update = vec_decrease(patch, ts)
            if is_corner_threshold(patch_update):
                input_data.append(patch_update.reshape(144,1))
                time_stamp.append(ts)

## OUTPUT
The output is a 2 x 1 vector composed of the components of the speed (vx, vy).
Thus we create the output vector, each time surface corresponds to one (vx, vy) vector. However, all the output correspoonding to time surfaces which are not corners must be removed from the set (second for-loop).

In [ ]:
print('Output being prepared')
output = []
i = 0
with open(output_file, 'r') as events:
    for ev in events:
        ev = ev.split()
        for i in range(1000):
            output.append([float(ev[3]), float(ev[4])])


#Only keep the right output (corner)
print('Output to be kept')
output_data = []

for i in range(len(output)):
    if len(time_stamp) == 0:
        break
    elif i == time_stamp[0]:
        output_data.append(output[i])
        del time_stamp[0]

## SAVE DATA

In [ ]:
output_data = np.asarray (output_data)
input_data = np.asarray (input_data)
input_data = np.squeeze(input_data, axis=2)

np.save('output_data.npy', output_data)
np.save('input_data.npy', input_data)

## Short Overview

In [ ]:
for i in range(20):
    plt.imshow(input_data[i].reshape(12,12), cmap="gray", vmin = 0, vmax =1)
    plt.show()